In [13]:
from dotenv import load_dotenv
import os
import pandas as pd
from typing import TypeVar, Any
from pydantic import BaseModel, Field, create_model

import litellm
from litellm import completion
from instructor import from_litellm, Mode



In [14]:
litellm.drop_params = True  # watsonx.ai doesn't support `json_mode`
client = from_litellm(completion, mode=Mode.JSON)  # create an instructor client from litellm

In [ ]:
# create a response model
class Response(BaseModel): # <--- BaseModel is a Pydantic class
    """The predicted emotion from a customer review."""
    answer: str = Field(..., description="One of Ekman's 7 emotions")

In [16]:

# define a prompt
prompt = """You are an emotion detection expert. Identify the emotion in the text and return it as a string.
"""

# make a request to the LLM
response = client.chat.completions.create( 
            model="watsonx/mistralai/mistral-large", 
            messages=[
                {
                    "role": "user",
                    "content": prompt, 
                }
            ],
            project_id=os.getenv("WX_PROJECT_ID_RAG"), 
            apikey=os.getenv("WX_API_KEY"),
            api_base=os.getenv("WX_API_URL"),
            response_model=Response, 
)

In [17]:
response_model = create_model(
    "MyResponseModel", 
    reasoning=(str, Field(description="The short reasoning behind the answer")),
    answer=(str, Field(description="Your answer to the question")),
    __base__=BaseModel
) 

In [ ]:
class BaseResponse(BaseModel):
    """A default response model that stores a list of predicted Ekman emotions. We will use this to predict the emotions of a review."""
    answer: str

ResponseType = TypeVar("ResponseType", bound=BaseModel)

class LLMCaller:
    """
    A class to interact with a Large Language Model (LLM)
    using the LiteLLM and Instructor libraries.
    
    Designed to send prompts and receive structured responses
    as Pydantic models (e.g., predicted emotions).
    """
    def __init__(self, api_key: str, project_id: str, api_url: str, model_id: str, params: dict[str, Any]):
        """Initializes the LLMCaller with Watsonx credentials and configuration."""
        self.api_key = api_key
        self.project_id = project_id
        self.api_url = api_url
        self.model_id = model_id
        self.params = params

        litellm.drop_params = True
        self.client = from_litellm(completion, mode=Mode.JSON)

    def create_response_model(self, title: str, fields: dict) -> ResponseType:
        """ Dynamically creates a Pydantic response model for the LLM's output.
        Args:
            title (str): The name of the response model.
            fields (dict): A dictionary defining the fields of the response model.
                           Keys are field names, and values are tuples of (type, Field).

        Returns:
            ResponseType: A dynamically created Pydantic model class.
        """
        return create_model(title, **fields, __base__=BaseResponse)

    def invoke(self, prompt: str, response_model: ResponseType = BaseResponse, **kwargs) -> ResponseType:
        """ Sends a prompt to the LLM and retrieves a structured response.

        Args:
            prompt (str): The input prompt to send to the LLM.
            response_model (ResponseType): The Pydantic model to structure the LLM's response.
                                           Defaults to BaseResponse.
            **kwargs: Additional arguments to pass to the LLM client.

        Returns:
            ResponseType: The structured response from the LLM, parsed into the specified response model.
        """
        response = self.client.chat.completions.create(
            model=self.model_id,
            messages=[{
                "role": "user",
                "content": prompt + "\n\nRespond using this structure: " + str(response_model.__annotations__)
            }],
            project_id=self.project_id,
            apikey=self.api_key,
            api_base=self.api_url,
            response_model=response_model,
            **kwargs
        )
        return response



In [19]:
load_dotenv()

llm = LLMCaller(
    api_key=os.getenv("WX_API_KEY"),
    project_id=os.getenv("WX_PROJECT_ID_RAG"),
    api_url=os.getenv("WX_URL"),
    model_id="watsonx/mistralai/mistral-large",
    params={"max_tokens": 100}
)


In [20]:
# Load the emotion definitions
df_defs = pd.read_csv("data/oxford_ekman_emotions.csv")
emotion_definitions = dict(zip(df_defs["emotion"], df_defs["definition"]))



In [21]:
class EmotionResponse(BaseModel):
    emotions: list[str] = Field(..., description="The list of Ekman emotions expressed in the review.")


In [24]:
llm.invoke("im sad", response_model=EmotionResponse) 

EmotionResponse(emotions=['sad'])

In [ ]:
response = llm.invoke(
    prompt="im sad", 
    response_model=llm.create_response_model(  # create a response model dynamically
        "EmotionResponse", 
        {
            "reasoning": (str, Field(...)),
            "Emotion": (str, Field
                (
                    ...,
                    description="The emotion of the review."
                )
            )
        }
    )
)

print(response.answer)
print(response.reasoning)

I'm sorry to hear that you're feeling sad. How can I help you?
Sadness is a natural emotion, and it's important to acknowledge and address it. Offering support is a first step to helping you feel better or seek the help you need if necessary. Expressing empathy and providing an opening for further conversation can often be comforting and helpful in understanding the reasons behind the sadness and finding ways to address them effectively. By doing so, I'm supporting your emotional well-being and aiming to connect with you in a meaningful way that may lead to a positive resolution or, at the very least, a sense of being heard and understood. If the feeling persists or worsens, it's also important to consider reaching out to a mental health professional for further assistance and guidance. Mental health and emotional well-being are crucial aspects of overall well-being, and professional help can often provide valuable insights and strategies to cope with and manage feelings of sadness and

Prompt sent to LLM:
You are a helpful assistant that classifies customer reviews using Ekman's 7 emotions.
Use ONLY the following emotion definitions:

ANGER: A strong feeling of annoyance, displeasure, or hostility.
DISGUST: A strong feeling of dislike or disapproval for something unpleasant or offensive.
FEAR: An unpleasant emotion caused by the threat of danger, pain, or harm.
JOY: A feeling of great pleasure and happiness.
SADNESS: The condition or quality of being sad; sorrow; a feeling of unhappiness or grief.
SURPRISE: A feeling of mild astonishment or shock caused by something unexpected.
NEUTRAL: Not displaying any strong emotion or feeling; a lack of emotional expression.


Classify the following review into one or more Ekman emotions:

Review: "I was furious at the lack of help and felt completely disrespected."
List all matching emotions. If none apply, return ['neutral'].
Predicted emotions: ['ANGER', 'DISGUST']
